In [1]:
!pip install phe

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
import time
import psutil
import threading
import matplotlib.pyplot as plt
import os
from threading import Thread
import pandas as pd
plt.style.use("dark_background")

In [3]:
X_train1=pd.read_csv('X_train2.csv')
X_train=X_train1.to_numpy()
print(X_train.shape)
y_train1=pd.read_csv('y_train2.csv')
y_train=y_train1.to_numpy()
print(y_train.shape)
X_test1=pd.read_csv('X_test2.csv')
X_test=X_test1.to_numpy()
print(X_test.shape)
y_test1=pd.read_csv('y_test2.csv')
y_test=y_test1.to_numpy()
print(y_test.shape)

(640, 12)
(640, 1)
(160, 12)
(160, 1)


In [4]:
X_train.shape

(640, 12)

In [5]:
X_train_augmented = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
X_train_augmented

array([[1.000e+00, 1.300e+02, 2.002e+03, ..., 4.270e-01, 1.000e+01,
        4.000e+01],
       [1.000e+00, 7.100e+01, 2.013e+03, ..., 8.920e-01, 1.390e+01,
        0.000e+00],
       [1.000e+00, 0.000e+00, 2.014e+03, ..., 4.760e-01, 1.000e+01,
        8.600e+01],
       ...,
       [1.000e+00, 8.000e+00, 2.010e+03, ..., 7.370e-01, 1.170e+01,
        6.000e+00],
       [1.000e+00, 3.600e+01, 2.008e+03, ..., 6.980e-01, 1.300e+01,
        9.000e+00],
       [1.000e+00, 5.000e+00, 2.005e+03, ..., 6.790e-01, 1.090e+01,
        1.000e+00]])

In [6]:
X_train_T = X_train_augmented.T

In [7]:
noise=1234
X_train_T1=X_train_T*noise

In [8]:
from phe import paillier
public_key,private_key=paillier.generate_paillier_keypair()

In [9]:
l=0
enc_X_train=[]
for i in X_train_T:
    row_in=[]
    for j in i:
        p=public_key.encrypt(j)
        row_in.append(p)
    enc_X_train.append(row_in)
    print(f"row {l} encrypted")
    l=l+1

row 0 encrypted
row 1 encrypted
row 2 encrypted
row 3 encrypted
row 4 encrypted
row 5 encrypted
row 6 encrypted
row 7 encrypted
row 8 encrypted
row 9 encrypted
row 10 encrypted
row 11 encrypted
row 12 encrypted


In [10]:
len(row_in)

640

In [11]:
len(X_train_T)

13

In [12]:
enc_y_train=[]
for i in y_train:
    p=public_key.encrypt(i[0])
    enc_y_train.append(p)

In [13]:
def get_cpu_and_ram_utilization(pid, duration):
    process = psutil.Process(pid)
    cpu_usages = []
    ram_usages = []
    start_time = time.time()
    while time.time() - start_time < duration:
        cpu_usage = process.cpu_percent(interval=1)
        ram_usage = process.memory_percent()
        cpu_usages.append(cpu_usage)
        ram_usages.append(ram_usage)
        print(f"CPU utilization: {cpu_usage}% | RAM utilization: {ram_usage}%")
    total_cpu_usage = sum(cpu_usages)
    return total_cpu_usage, cpu_usages, ram_usages

In [14]:
pid = os.getpid()
monitor_duration = 1200

In [15]:
time.sleep(15)

In [16]:
results={}

def train_lin_reg(enc_X_train, X_train_T1, enc_y_train):
  train_start=time.time()
  XTX=[]
  for i in enc_X_train:
      row_in=[]
      for j in X_train_T1:
          row_in2=0
          for p,q in zip(i,j):
              t=p*q
              row_in2=row_in2+t
          row_in.append(row_in2)
      XTX.append(row_in)

  XTY=[]
  for i in X_train_T1:
      summation=0
      for p,q in zip(i,enc_y_train):
          t=p*q
          summation=summation+t
      XTY.append(summation)

  XTX_dec=[]
  for i in XTX:
      row_in=[]
      for j in i:
          p=private_key.decrypt(j)
          p=p/noise
          row_in.append(p)
      XTX_dec.append(row_in)

  XTX_dec=np.array(XTX_dec)

  XTY_dec=[]
  for i in XTY:
      p=private_key.decrypt(i)
      p=p/noise
      XTY_dec.append(p)

  XTY_dec=np.array(XTY_dec)

  XTX_inv = np.linalg.inv(XTX_dec)
  theta = np.dot(XTX_inv, XTY_dec)

  results['bias'] = theta[0]
  results['weights'] = theta[1:]

  train_end=time.time()

  results['train_time']=train_end-train_start
  print(results['train_time'])


In [17]:
train_thread = Thread(target=train_lin_reg, args=(enc_X_train, X_train_T1, enc_y_train))
train_thread.start()

total_cpu_usage, cpu_usages, ram_usages = get_cpu_and_ram_utilization(pid, monitor_duration)
train_thread.join()

print(f"Total CPU utilization over {monitor_duration} seconds: {total_cpu_usage}%")

# Print the CPU and RAM usage per second
print("CPU usage per second:", cpu_usages)
print("RAM usage per second:", ram_usages)

# Calculate total CPU resource consumption in 'CPU-seconds'
cpu_seconds = sum(cpu_usages) / 100
print(f"Total CPU resource consumption: {cpu_seconds} CPU-seconds")

CPU utilization: 95.2% | RAM utilization: 1.6487916896781603%
CPU utilization: 100.2% | RAM utilization: 1.6487916896781603%
CPU utilization: 98.4% | RAM utilization: 1.6487916896781603%
CPU utilization: 99.2% | RAM utilization: 1.6487916896781603%
CPU utilization: 99.1% | RAM utilization: 1.6487916896781603%
CPU utilization: 100.2% | RAM utilization: 1.6487916896781603%
CPU utilization: 101.8% | RAM utilization: 1.6487916896781603%
CPU utilization: 102.6% | RAM utilization: 1.6487916896781603%
CPU utilization: 100.9% | RAM utilization: 1.6487916896781603%
CPU utilization: 102.6% | RAM utilization: 1.6487916896781603%
CPU utilization: 99.4% | RAM utilization: 1.6487916896781603%
CPU utilization: 97.0% | RAM utilization: 1.6487916896781603%
CPU utilization: 97.4% | RAM utilization: 1.6487916896781603%
CPU utilization: 98.1% | RAM utilization: 1.6487916896781603%
CPU utilization: 100.0% | RAM utilization: 1.6487916896781603%
CPU utilization: 102.1% | RAM utilization: 1.6487916896781603%


In [18]:
bias=results['bias']
bias

-1.8007023181211825

In [19]:
weights=results['weights']
weights

array([-8.20534727e-03,  3.14017771e-02, -1.72825878e+00, -2.58487151e-02,
        4.44136399e-01, -1.23342549e-01,  3.12011913e-05,  4.53284482e-05,
        1.64899037e-03,  1.04763172e+01,  7.25523374e-01, -3.76078791e-01])

In [20]:
X_test[0]

array([0.00000000e+00, 2.00500000e+03, 1.00000000e+00, 2.91000000e+02,
       8.50000000e+01, 2.00000000e-02, 1.38864773e+00, 2.52941299e+01,
       2.58000000e+00, 3.96000000e-01, 7.90000000e+00, 1.18000000e+02])

In [21]:
enc_X_test=[]
for i in X_test:
    row_in=[]
    for j in i:
        p=public_key.encrypt(j)
        row_in.append(p)
    enc_X_test.append(row_in)

In [22]:
predictions=[]
test_start=time.time()
for i in enc_X_test:
    pred=0
    for j,k in zip(i,weights):
        p=j*k
        pred=pred+p
    pred=pred+bias
    predictions.append(pred)
test_end=time.time()
test_time=test_end-test_start


In [23]:
test_time

23.577752828598022

In [24]:
results['train_time']

968.5825083255768

In [25]:
unenc_preds=[]
for i in predictions:
  unenc_preds.append(private_key.decrypt(i))

unenc_preds

[55.16715610321978,
 70.36491712058506,
 52.97268241040151,
 68.30201694005044,
 70.40541632907659,
 73.62230303952897,
 55.10561545907716,
 61.12499742539327,
 71.0469409144975,
 72.10252095689549,
 68.24374153213626,
 70.50009216489462,
 78.52095733146213,
 72.76816440645699,
 68.51145742290916,
 66.81414697928503,
 59.00972952375638,
 66.88137343431255,
 69.22207146849165,
 75.89716886625382,
 66.43794628689928,
 47.30714873825693,
 73.78476335890058,
 56.4854061804599,
 66.53748034983184,
 84.00130584311273,
 72.11350594764232,
 81.93529725640953,
 59.148252050946084,
 70.18505260748289,
 72.14801133056616,
 75.83883381241375,
 73.20111442445686,
 69.20090235788786,
 77.1042171891403,
 72.44238000223568,
 72.6787380795096,
 65.32132719063063,
 75.1248888228369,
 69.10436888627483,
 77.54928301425385,
 72.92825287605764,
 74.73265704187467,
 81.12132710807761,
 66.97585883406501,
 67.27118289875845,
 74.02966895788533,
 68.3713604838646,
 70.68155196017483,
 70.26079168337067,
 57.2

In [26]:
!pip install joblib

In [27]:
import joblib
joblib.dump((weights, bias, unenc_preds, cpu_seconds, cpu_usages,ram_usages, results['train_time'], test_time), 'variables_paillier.pkl')

['variables_paillier.pkl']

In [ ]:
time.sleep(11600)